In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import box
import psycopg2
import re
import os
from dotenv import load_dotenv
load_dotenv()
conn = psycopg2.connect(f"dbname={os.getenv('SQL_DATABASE')} user={os.getenv('SQL_DATABASE')} password={os.getenv('SQL_PASSWORD')} host={os.getenv('SQL_HOST')} port={os.getenv('SQL_PORT')}")
from sqlalchemy import create_engine

engine = create_engine(f"postgresql+psycopg2://{os.getenv('SQL_USER')}:{os.getenv('SQL_PASSWORD')}@{os.getenv('SQL_HOST')}:{os.getenv('SQL_PORT')}/{os.getenv('SQL_DATABASE')}")

model_coeff_pond = 0.705*0.729 # model v6 corrections stats detections config ponderation FP Parcel x0.705, ponderation ratios constructions légitimes x 0.729


herault_base_params = {
    'commune_batch_id':36513,
    'geozone_id': 262,
    'ze_zone_id': 351,
    'zr_zone_id': 352,
    'zan_zone_id': 361,
    'batch_id_year_0': 217,
    'batch_id_year_3': 222,
    'batch_id_year_6': 223,
    'millesime_year_0': 2018,
    'millesime_year_3': 2021,
    'millesime_year_6': 2024
}


# 1. aggregate detections datas

In [ ]:
query = f"""

-- Step 2: Inserer les données calculées et agrégées
WITH params AS (
        SELECT %(geozone_id)s AS commune_geozone_id,
        %(commune_batch_id)s AS commune_batch_id,
        %(batch_id_year_0)s AS batch_id_year_0,
        %(batch_id_year_3)s AS batch_id_year_3,
        %(batch_id_year_6)s AS batch_id_year_6,
        %(millesime_year_0)s AS millesime_year_0,
        %(millesime_year_3)s AS millesime_year_3,
        %(millesime_year_6)s AS millesime_year_6,
        %(ze_zone_id)s AS ze_zone_id,
        %(zr_zone_id)s AS zr_zone_id,
        %(zan_zone_id)s AS zan_zone_id
),
batch_data_to_insert AS 
(
SELECT DISTINCT ON (inf.id)
    inf.id,
    inf.score,
    CASE 
        WHEN inf.object_type IN ('construction en dur', 'construction legere yourte etc.', 'container', 'installation legere') THEN 'construction en dur'
        ELSE inf.object_type
    END AS object_type,
    inf.geometry,
    p.commune_geozone_id AS geozone_id,
    p.commune_batch_id AS commune_batch_id,
    CASE 
        WHEN p.commune_batch_id = p.batch_id_year_0 THEN p.millesime_year_0
        WHEN p.commune_batch_id = p.batch_id_year_3 THEN p.millesime_year_3
        WHEN p.commune_batch_id = p.batch_id_year_6 THEN p.millesime_year_6
        ELSE null
    END as batch_year,
    cp.id_parcellaire
FROM
    detections.inference inf 
    JOIN params p ON inf.batch_id = p.commune_batch_id
    LEFT JOIN core_parcel cp 
        ON ST_Intersects(cp.geometry, inf.geometry)
WHERE
    inf.score > 0.3
    and
    ST_Within(ST_SetSRID(inf.geometry, 4326), (
        SELECT geozone.geometry
        FROM core_geozone geozone
        WHERE geozone.id = p.commune_geozone_id
    ))
    AND inf.object_type IN (
        'construction en dur',
        'piscine',
        'mobil home',
        'caravane',
        'construction legere yourte etc.',
        'container',
        'installation legere'
    )
    and 
   (
   ST_Within(ST_SetSRID(inf.geometry, 4326), (
        SELECT geozone.geometry
        FROM core_geozone geozone
        WHERE geozone.id = p.ze_zone_id -- zone enjeu env 
    ))
   or 
   ST_Within(ST_SetSRID(inf.geometry, 4326), (
        SELECT geozone.geometry
        FROM core_geozone geozone
        WHERE geozone.id = p.zr_zone_id -- zone risque fort
    ))
    or 
   ST_Within(ST_SetSRID(inf.geometry, 4326), (
        SELECT geozone.geometry
        FROM core_geozone geozone
        WHERE geozone.id = p.zan_zone_id -- zone naturelle et agricole
    ))
   )
   ORDER BY inf.id, cp.id  -- garder seulement la premiere parcelle associée a chaque detection
)
INSERT INTO detections.stats_detections_epci_analysis (
    id,
    score,
    object_type,
    geometry,
    geozone_id,
    batch_year,
    id_parcellaire,
    first_detection
)
SELECT 
    t.id,
    t.score,
    t.object_type,
    t.geometry,
    t.geozone_id,
    t.batch_year,
    t.id_parcellaire,
    NOT EXISTS (
        SELECT 1
        FROM detections.stats_detections_epci_analysis existing
        WHERE 
            ST_Intersects(t.geometry, existing.geometry)
            AND ST_Area(ST_Intersection(t.geometry, existing.geometry)) / ST_Area(t.geometry) > 0.5
    ) AS first_detection
FROM batch_data_to_insert t;
;
"""


In [ ]:
herault_stats_configs = [
    {'name':'CC Lodévois et Larzac', 'geozone_id': 36513,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CC du Grand Pic Saint-Loup', 'geozone_id': 36519,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CC Grand Orb communauté de communes en Languedoc', 'geozone_id': 36716,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CC Sud-Hérault', 'geozone_id': 36717,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CC du Minervois au Caroux', 'geozone_id': 36802,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CA Sète Agglopôle Méditerranée', 'geozone_id': 36803,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CC Les Avant-Monts', 'geozone_id': 37087,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'Montpellier Méditerranée Métropole', 'geozone_id': 37372,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CC du Clermontais', 'geozone_id': 37373,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CA du Pays de l Or', 'geozone_id': 37374,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CC La Domitienne', 'geozone_id': 37375,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CA Lunel Agglo', 'geozone_id': 37376,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CC Vallée de l Hérault', 'geozone_id': 37377,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CC des Cévennes Gangeoises et Suménoises', 'geozone_id': 37378,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CA de Béziers-Méditerranée', 'geozone_id': 37379,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223},
    {'name':'CA Hérault-Méditerranée', 'geozone_id': 37380,'batch_id_year_0': 217, 'batch_id_year_3': 222,'batch_id_year_6': 223}
]

for config in herault_stats_configs:
    print(f"Processing : {config['name']} - {config['geozone_id']} ")
    run_params = herault_base_params.copy()
    run_params['geozone_id'] = config['geozone_id']
    run_params['batch_id_year_0'] = config['batch_id_year_0']
    run_params['batch_id_year_3'] = config['batch_id_year_3']
    run_params['batch_id_year_6'] = config['batch_id_year_6']
    print(f"")
    for batch_id in [run_params['batch_id_year_0'],run_params['batch_id_year_3'],run_params['batch_id_year_6']]:
        run_params['commune_batch_id'] = batch_id
        print(f"batch : {run_params['commune_batch_id']}")
        print(run_params)
        conn = psycopg2.connect(f"dbname={os.getenv('SQL_DATABASE')} user={os.getenv('SQL_DATABASE')} password={os.getenv('SQL_PASSWORD')} host={os.getenv('SQL_HOST')} port={os.getenv('SQL_PORT')}")
        cursor = conn.cursor()
        cursor.execute(query, run_params)
        conn.commit()
        cursor.close()
        conn.close()


# 2. extract pv from lucca exports

In [ ]:
pv_lucca_herault_df = pd.read_excel(r"E:\Téléchargements\20250225_PV-CABANISATION_HERAULT.xlsx")
def normalize_date_string(date_str):
    match = re.match(r"(\d{1,2})/(\d{1,2})/(\d{2})$", date_str)
    if match:
        # It's a 2-digit year, convert to 4-digit (assume 2000s)
        day, month, year = match.groups()
        year = "20" + year
        return f"{year}-{month}-{day} 00:00:00"
    return date_str  # Already in 4-digit format or malformed
pv_lucca_herault_df["DATE_normalized"] = pv_lucca_herault_df["DATE"].astype(str).apply(normalize_date_string)
pv_lucca_herault_df['status_date'] = pd.to_datetime(pv_lucca_herault_df['DATE_normalized'],format="%Y-%m-%d %H:%M:%S",errors='coerce')
pv_lucca_herault_df = pv_lucca_herault_df[~pv_lucca_herault_df.status_date.isna()]
pv_lucca_herault_df['status_year'] = pv_lucca_herault_df['status_date'].dt.year.astype(int)
pv_lucca_herault_df['COMMUNE_NAME']= pv_lucca_herault_df['COMMUNE'].str.replace(' ','-')
pv_lucca_herault_df['COMMUNE']= pv_lucca_herault_df['COMMUNE'].str.replace(' ','-')
pv_lucca_herault_df['COMMUNE']= pv_lucca_herault_df['COMMUNE'].str.upper()
pv_lucca_herault_df['COMMUNE']= pv_lucca_herault_df['COMMUNE'].str.replace('_','-')
pv_lucca_herault_df['COMMUNE']= pv_lucca_herault_df['COMMUNE'].str.replace("'",'-')
pv_lucca_herault_df['COMMUNE']= pv_lucca_herault_df['COMMUNE'].str.replace("-/-",'-')
pv_lucca_herault_df['COMMUNE']= pv_lucca_herault_df['COMMUNE'].str.replace("’",'-')
pv_lucca_herault_df['COMMUNE']= pv_lucca_herault_df['COMMUNE'].str.replace("CAP-D-AGDE",'AGDE')
pv_lucca_herault_df['COMMUNE']= pv_lucca_herault_df['COMMUNE'].str.replace("MONS-LA-TRIVALLE",'MONS')
pv_lucca_herault_df['COMMUNE']= pv_lucca_herault_df['COMMUNE'].str.replace("SAINT-CHRISTOL-ENTRE-VIGNES",'ENTRE-VIGNES')
pv_lucca_herault_df['COMMUNE']= pv_lucca_herault_df['COMMUNE'].str.replace("VERARGUES-ENTRE-VIGNES",'ENTRE-VIGNES')
pv_lucca_herault_df['COMMUNE']= pv_lucca_herault_df['COMMUNE'].str.replace("SAINT-JEAN-DU-MINERVOIS",'SAINT-JEAN-DE-MINERVOIS')
pv_lucca_herault_df['COMMUNE']= pv_lucca_herault_df['COMMUNE'].str.replace("SAINT-JULIEN-D-ORLARGUES",'SAINT-JULIEN')
pv_lucca_herault_df.loc[
    pv_lucca_herault_df['COMMUNE'] == 'VACQUERIE-ET-ST-MARTIN-DE-CASTRIES',
    'COMMUNE'
] = 'LA-VACQUERIE-ET-SAINT-MARTIN-DE-CASTRIES'
pv_lucca_herault_df.loc[
    pv_lucca_herault_df['COMMUNE'] == 'VALRAS',
    'COMMUNE'
] = 'VALRAS-PLAGE'
pv_lucca_herault_df.loc[
    pv_lucca_herault_df['COMMUNE'] == 'SALVETAT-SUR-AGOUT',
    'COMMUNE'
] = 'LA-SALVETAT-SUR-AGOUT'


pv_lucca_herault_df

In [ ]:
pv_summary_df = pv_lucca_herault_df[pv_lucca_herault_df.STATUS=='PV dressé'].groupby(['COMMUNE','CODE_INSEE','status_year']).size().reset_index()
pv_summary_df.rename(columns={0:'nb_pv'},inplace=True)

pv_summary_df

In [ ]:
df_communes = pd.read_sql("select gz.id, gz.name, gz.name_normalized, iso_code, department_id from public.core_geozone gz left join public.core_geocommune gc on gz.id = gc.geozone_ptr_id", con=conn)
df_communes = df_communes[df_communes.department_id==2.0]
df_communes['COMMUNE'] = df_communes['name_normalized'].str.upper()
df_communes['COMMUNE'] = df_communes['COMMUNE'].str.replace(' ','-')

df_communes.head()

In [ ]:
df_pv_to_insert = pd.merge(pv_summary_df,df_communes,how='left',left_on='COMMUNE',right_on='COMMUNE')
df_pv_to_insert

In [ ]:
df_pv_to_insert = df_pv_to_insert[['id','status_year','nb_pv']].rename(columns={'id':'geozone_id','status_year':'year'})
df_pv_to_insert['id'] = [x+55 for x in range(len(df_pv_to_insert))]
df_pv_to_insert

In [ ]:
df_pv_to_insert.to_sql('stats_pv_analysis', schema='analytics',con=engine,if_exists='append', index=False)

# 3. insert nb parcels at risk

In [ ]:

query_parcels = """
WITH 
params AS (
        SELECT %(geozone_id)s AS epci_geozone_id,
        %(stats_geozone_id)s AS stats_epci_geozone_id,
        %(ze_zone_id)s AS ze_zone_id,
        %(zr_zone_id)s AS zr_zone_id,
        %(zan_zone_id)s AS zan_zone_id
    ),
results_count_parcels as (
	SELECT cgep.id as "geozone_id", 
		count(distinct cp.id_parcellaire) as nb_parcels
		FROM
		core_geozone cg
		left join public.core_geocommune cgcommune on cg.id = cgcommune.geozone_ptr_id 

		left join public.core_geozone cgcom on cgcommune.geozone_ptr_id = cgcom.id
		left join public.core_geozone cgep on cgcommune.epci_id = cgep.id
		LEFT JOIN core_parcel cp on cg.id = cp.commune_id
		JOIN params p ON TRUE
		WHERE 
		cgep.id = p.epci_geozone_id 
		group by cgep.id
	),
result_count_ze_parcels as (
	-- Second query: Count of parcels within ZE zones for one commune
	SELECT 
	    cgep.id as "geozone_id",
	    COUNT(DISTINCT cp.id_parcellaire) AS nb_parcels_in_ze
		FROM
		core_geozone cg
		left join public.core_geocommune cgcommune on cg.id = cgcommune.geozone_ptr_id 
		left join public.core_geozone cgcom on cgcommune.geozone_ptr_id = cgcom.id
		left join public.core_geozone cgep on cgcommune.epci_id = cgep.id
		LEFT JOIN core_parcel cp on cg.id = cp.commune_id
		JOIN params p ON TRUE
		WHERE 
		cgep.id = p.epci_geozone_id 
		and 
		(
	    ST_Within(cp.geometry, (SELECT geometry FROM core_geozone WHERE id = p.ze_zone_id))
		 OR ST_Within(cp.geometry, (SELECT geometry FROM core_geozone WHERE id = p.zr_zone_id))
		 OR ST_Within(cp.geometry, (SELECT geometry FROM core_geozone WHERE id = p.zan_zone_id))
	 	)
	GROUP BY cgep.id
	)
INSERT INTO detections.stats_epci_parcels_analysis(geozone_id,nb_parcels,nb_parcels_in_ze)
select p.stats_epci_geozone_id, rcp.nb_parcels, rczp.nb_parcels_in_ze from result_count_ze_parcels rczp
left join results_count_parcels rcp on rczp.geozone_id = rcp.geozone_id
JOIN params p ON TRUE
;
"""

In [ ]:
herault_stats_configs = [
    {'name':'CC Lodévois et Larzac', 'geozone_id': 36513,'stats_geozone_id': 36513,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CC du Grand Pic Saint-Loup', 'geozone_id': 36519,'stats_geozone_id': 36519,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CC Grand Orb communauté de communes en Languedoc', 'geozone_id': 36716,'stats_geozone_id': 36716,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CC Sud-Hérault', 'geozone_id': 36717,'stats_geozone_id': 36717,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CC du Minervois au Caroux', 'geozone_id': 36802,'stats_geozone_id': 36802,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CA Sète Agglopôle Méditerranée', 'geozone_id': 36803,'stats_geozone_id': 36803,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CC Les Avant-Monts', 'geozone_id': 37087,'stats_geozone_id': 37087,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'Montpellier Méditerranée Métropole', 'geozone_id': 37372,'stats_geozone_id': 37372,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CC du Clermontais', 'geozone_id': 37373,'stats_geozone_id': 37373,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CA du Pays de l Or', 'geozone_id': 37374,'stats_geozone_id': 37374,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CC La Domitienne', 'geozone_id': 37375,'stats_geozone_id': 37375,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CA Lunel Agglo', 'geozone_id': 37376,'stats_geozone_id': 37376,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CC Vallée de l Hérault', 'geozone_id': 37377,'stats_geozone_id': 37377,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CC des Cévennes Gangeoises et Suménoises', 'geozone_id': 37378,'stats_geozone_id': 37378,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CA de Béziers-Méditerranée', 'geozone_id': 37379,'stats_geozone_id': 37379,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361},
    {'name':'CA Hérault-Méditerranée', 'geozone_id': 37380,'stats_geozone_id': 37380,'ze_zone_id': 351, 'zr_zone_id': 352,'zan_zone_id': 361}
]


for config in herault_stats_configs:
    print(f"Processing : {config['name']} - {config['geozone_id']} ")
    run_params = config.copy()
    run_params.pop('name',None)
    print(run_params)
    conn = psycopg2.connect(f"dbname={os.getenv('SQL_DATABASE')} user={os.getenv('SQL_DATABASE')} password={os.getenv('SQL_PASSWORD')} host={os.getenv('SQL_HOST')} port={os.getenv('SQL_PORT')}")
    cursor = conn.cursor()
    cursor.execute(query_parcels, run_params)
    conn.commit()
    cursor.close()
    conn.close()

# 4. insert nb status set to legitimate 

In [ ]:
"""

-- Step 4: Insert data into the table [re run on each tileset and communes geozones id, if no year info set "year manually" dividing by frequency of tilesets]
INSERT INTO analytics.stats_status_analysis (geozone_id, "year", nb_legitimate_status)
WITH data_status_analysis AS (
    SELECT 
        cg2.id, 
        cg2.name, 
        cg.iso_code, 
        cdd.detection_validation_status, 
        COUNT(*) AS nb_legitimate_status
    FROM public.core_detection cd
    LEFT JOIN public.core_detectionobject cdo ON cd.detection_object_id = cdo.id
    LEFT JOIN public.core_detectiondata cdd ON cd.detection_data_id = cdd.id
    LEFT JOIN public.tileset_enriched te ON cd.tile_set_id = te.id 
    LEFT JOIN public.core_geocommune cg ON cdo.commune_id = cg.geozone_ptr_id
    LEFT JOIN public.core_geozone cg2 ON cg.geozone_ptr_id = cg2.id 
    WHERE te.id = 2 
      AND cdd.detection_validation_status = 'LEGITIMATE'
      AND cg2.id IN (262, 333, 196, 264, 218, 312, 112, 62, 52, 141)
    GROUP BY cg2.id, cg2.name, cg.iso_code, cdd.detection_validation_status
)
SELECT 
    id AS geozone_id,
    2021 AS "year",
    nb_legitimate_status/3
FROM data_status_analysis;
"""

# 5. insert nb controls analytics

In [ ]:
"""

-- Step 2: Insert data into the table [re run on each tileset and communes geozones id, if no year info set "year manually" dividing by frequency of tilesets]
INSERT INTO analytics.stats_control_analysis (geozone_id, "year", nb_controls)
WITH data_control_analysis AS (
    SELECT 
        cg2.id, 
        cg2.name, 
        cg.iso_code, 
        cdd.detection_control_status, 
        COUNT(*) AS nb_controls
    FROM public.core_detection cd
    LEFT JOIN public.core_detectionobject cdo ON cd.detection_object_id = cdo.id
    LEFT JOIN public.core_detectiondata cdd ON cd.detection_data_id = cdd.id
    LEFT JOIN public.tileset_enriched te ON cd.tile_set_id = te.id 
    LEFT JOIN public.core_geocommune cg ON cdo.commune_id = cg.geozone_ptr_id
    LEFT JOIN public.core_geozone cg2 ON cg.geozone_ptr_id = cg2.id 
    WHERE te.id = 1 
      AND cdd.detection_control_status = 'CONTROLLED_FIELD'
      AND cg2.id IN (262, 333, 196, 264, 218, 312, 112, 62, 52, 141)
    GROUP BY cg2.id, cg2.name, cg.iso_code, cdd.detection_control_status
)
SELECT 
    id AS geozone_id,
    2021 AS "year",
    nb_controls/3
FROM data_control_analysis;
"""

# 6. create the base view to monitor evolution

In [ ]:
# todo in sql_scripts/stats_cabanisation_progression.sql

# 7. data exploration for dashboards

In [ ]:
df_data = pd.read_sql("""select sce.*, spa.nb_parcels, spa.nb_parcels_in_ze from analytics.stats_parcels_analysis spa left join analytics.stats_cabanisation_evolution sce on spa.geozone_id = sce.geozone_id;""",con=engine)
df_data[['department_name', 'geozone_code', 'geozone_id', 'nb_pv_2018_2020',
       'nb_pv_2021_2023', 'situation_parcelles_cabanisees_2021',
       'situation_parcelles_cabanisees_2024',
       'nouvelles_parcelles_cabanisees_2024_vs_2021',
       'parcelles_remises_en_etat_2024_vs_2021', 'nb_parcels',
       'nb_parcels_in_ze']]

In [ ]:
df_data['score_commune_potentiel_cabanisation'] = df_data['nb_parcels_in_ze'] / df_data['nb_parcels']
df_data['score_action_cabanisation'] = (df_data['nb_pv_2021_2023']+df_data['nb_pv_2018_2020']) / df_data['situation_parcelles_cabanisees_2021']
df_data['score_action_parcel'] = (df_data['nb_pv_2021_2023']+df_data['nb_pv_2018_2020']) / df_data['nb_parcels_in_ze']
df_data['score_etat_cabanisation'] = df_data['situation_parcelles_cabanisees_2021'] / df_data['nb_parcels_in_ze']
df_data['score_nouvelles_cabanisation'] = df_data['nouvelles_parcelles_cabanisees_2024_vs_2021'] / df_data['nb_parcels_in_ze']
df_data['score_remises_en_etat'] = df_data['parcelles_remises_en_etat_2024_vs_2021'] / df_data['nb_parcels_in_ze']
df_data['score_evolution_cabanisation'] = (df_data['situation_parcelles_cabanisees_2024'] - df_data['situation_parcelles_cabanisees_2021']) / df_data['nb_parcels_in_ze']


In [ ]:
#df_data = df_data[df_data.department_name=='herault']
df_features = df_data[['score_commune_potentiel_cabanisation',
       'score_action_cabanisation', 'score_action_parcel',
       'score_etat_cabanisation', 'score_nouvelles_cabanisation',
       'score_remises_en_etat', 'score_evolution_cabanisation']]

correlation_matrix = df_features.corr()

In [ ]:
df_features

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()


In [ ]:
# vizualize spatial distribution of detections herault 2018

In [ ]:
gdf_2018_detections_in_ze = gpd.read_postgis(sql = """SELECT 
	--cd.tile_set_id, cd2.detection_validation_status, count(1)
	cd.tile_set_id,
    cd3.commune_id,
    cd3.parcel_id,
    ST_Centroid(cd.geometry)::geometry(Point, 4326) AS barycenter_detection,
    cd2.detection_validation_status
   FROM core_detection cd
     LEFT JOIN core_detectiondata cd2 ON cd.detection_data_id = cd2.id
     LEFT JOIN core_detectionobject cd3 ON cd.detection_object_id = cd3.id
     LEFT JOIN core_objecttype co ON cd3.object_type_id = co.id
     LEFT JOIN core_geozone cg_1 ON cd3.commune_id = cg_1.id
  WHERE (cd.tile_set_id = 2) 
  AND (co.name::text = ANY (ARRAY['Construction en dur'::character varying, 'Piscine'::character varying, 'Mobil-home'::character varying, 'Caravane'::character varying, 'Installation Legere'::character varying]::text[])) 
  AND (
  		st_within(st_setsrid(cd.geometry, 4326),( SELECT geozone.geometry FROM core_geozone geozone WHERE geozone.id = 351)) 
  		OR 
  		st_within(st_setsrid(cd.geometry, 4326), ( SELECT geozone.geometry FROM core_geozone geozone WHERE geozone.id = 352)) 
  		OR 
  		st_within(st_setsrid(cd.geometry, 4326), ( SELECT geozone.geometry FROM core_geozone geozone WHERE geozone.id = 361))
	  ) 
  AND (
  		(
		cd2.detection_validation_status::text = 'DETECTED_NOT_VERIFIED'::text 
  		AND cd.score > ((1::numeric - 0.3) * 0.3)::double precision 
  		AND (cd2.detection_prescription_status IS NULL OR cd2.detection_prescription_status::text <> 'PRESCRIBED'::text)
  		)
  		or 
  		(
  		cd2.detection_validation_status::text = 'LEGITIMATE'::text 
  		)
  		or 
  		(
  		cd2.detection_validation_status::text = 'SUSPECT'::text 
  		)
	  )""", con=conn, geom_col='barycenter_detection')
gdf_2018_detections_in_ze

In [ ]:
gdf_2018_detections_in_ze.to_file(r"E:\Datasets\aigle\aigle_results\20251008-aerial-herault-2018-ze\herault_2018_detections_ze.shp")

In [ ]:
gdf_2021_ddt_detections_in_ze = gpd.read_postgis(sql = """SELECT 
	--cd.tile_set_id, cd2.detection_validation_status, count(1)
	cd.tile_set_id,
    cd3.commune_id,
    cd3.parcel_id,
    cd.geometry,
    ST_Centroid(cd.geometry)::geometry(Point, 4326) AS barycenter_detection,
    cd2.detection_validation_status
   FROM core_detection cd
     LEFT JOIN core_detectiondata cd2 ON cd.detection_data_id = cd2.id
     LEFT JOIN core_detectionobject cd3 ON cd.detection_object_id = cd3.id
     LEFT JOIN core_objecttype co ON cd3.object_type_id = co.id
     LEFT JOIN core_geozone cg_1 ON cd3.commune_id = cg_1.id
     left join core_geocommune cg on cg_1.id = cg.geozone_ptr_id
  WHERE 
  	cd.tile_set_id = 1 
  	and 
  	(
  		(cg.iso_code in ('34002','34004','34006','34007','34009','34085','34098','34059','34160','34170','34190','34218','34228','34271','34284','34291','34290','34332') and 
  			(
  				(cd.score > ((1::numeric - 0.3) * 0.3)::double precision 
  				and cd2.detection_validation_status::text = 'DETECTED_NOT_VERIFIED'::text
  				and (cd2.detection_prescription_status IS NULL OR cd2.detection_prescription_status::text <> 'PRESCRIBED'::text)
  				)
  				or
		  		(
		  		cd2.detection_validation_status::text = 'LEGITIMATE'::text 
		  		)
		  		or 
		  		(
		  		cd2.detection_validation_status::text = 'SUSPECT'::text 
		  		)
	  		)
  		)
  		or
  		(cg.iso_code in ('34015','34021','34028','34053','34118','34224','34232','34279','34334','34335','34339') and 
  			(
  				(cd.score > ((1::numeric - 0.3) * 0.5)::double precision 
  				and cd2.detection_validation_status::text = 'DETECTED_NOT_VERIFIED'::text
  				and (cd2.detection_prescription_status IS NULL OR cd2.detection_prescription_status::text <> 'PRESCRIBED'::text)
  				)
  				or
		  		(
		  		cd2.detection_validation_status::text = 'LEGITIMATE'::text 
		  		)
		  		or 
		  		(
		  		cd2.detection_validation_status::text = 'SUSPECT'::text 
		  		)
	  		)
  		)
  		or
  		(cg.iso_code in ('34014','34022','34024','34043','34058','34066','34082','34123','34305','34153','34143','34165','34181','34247','34255','34309','34327') and
  			(
  				(cd.score > ((1::numeric - 0.3) * 0.6)::double precision 
  				and cd2.detection_validation_status::text = 'DETECTED_NOT_VERIFIED'::text
  				and (cd2.detection_prescription_status IS NULL OR cd2.detection_prescription_status::text <> 'PRESCRIBED'::text)
  				)
  				or
		  		(
		  		cd2.detection_validation_status::text = 'LEGITIMATE'::text 
		  		)
		  		or 
		  		(
		  		cd2.detection_validation_status::text = 'SUSPECT'::text 
		  		)
	  		)
  		)
	)
  AND (co.name::text = ANY (ARRAY['Construction en dur'::character varying, 'Piscine'::character varying, 'Mobil-home'::character varying, 'Caravane'::character varying, 'Installation Legere'::character varying]::text[])) 
  AND (
  		st_within(st_setsrid(cd.geometry, 4326),( SELECT geozone.geometry FROM core_geozone geozone WHERE geozone.id = 351)) 
  		OR 
  		st_within(st_setsrid(cd.geometry, 4326), ( SELECT geozone.geometry FROM core_geozone geozone WHERE geozone.id = 352)) 
  		OR 
  		st_within(st_setsrid(cd.geometry, 4326), ( SELECT geozone.geometry FROM core_geozone geozone WHERE geozone.id = 361))
	  ) """, con=conn, geom_col='geometry')
gdf_2021_ddt_detections_in_ze

In [ ]:
gdf_2021_ddt_detections_in_ze.to_file(r"E:\Datasets\aigle\aigle_results\20251008-aerial-herault-2018-ze\gdf_2021_ddt_detections_in_ze.shp")